# First draft of an algorithm

## Context

We want to reproduce the table from this [Wikipedia page](https://fr.wikipedia.org/wiki/Algorithme_de_Dijkstra).

## 1. Preparing the adjacency table

The table below represents the distances between nodes.

In [1]:
import numpy as np
import pandas as pd
wiki = pd.read_csv("wikipedia.csv", sep = ";")
adjacency_table = wiki.pivot(index='to', columns='from', values='distance')
adjacency_table

from,A,B,C,D,E,F,H,I
to,,,,,,,,
B,85.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
C,217.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
E,173.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
F,NaN,80.0,NaN,NaN,NaN,NaN,NaN,NaN
G,NaN,NaN,186.0,NaN,NaN,NaN,NaN,NaN
H,NaN,NaN,103.0,183.0,NaN,NaN,NaN,NaN
I,NaN,NaN,NaN,NaN,NaN,250.0,NaN,NaN
J,NaN,NaN,NaN,NaN,502.0,NaN,167.0,84.0


**Our objective is to find the shortest path between two nodes**. Let's call the node from which we start ```start_node```, and the one we want to reach ```end_node```.

In [2]:
start_node = "A"
end_node = "J"

**To find the shortest path, we will use the Dijkstra algorithm**. This implies building a Dijkstra table. Let's thus initialize a "Dijkstra" table.

In [3]:
# Building a dataframe
all_nodes = list(set(list(wiki["from"]) + list(wiki["to"])))
all_nodes.sort()
dijkstra_table = pd.DataFrame(columns=list(adjacency_table), index=list(adjacency_table.index))

# Filling this dataframe
for row_index in dijkstra_table.index:
    for col_index in list(dijkstra_table):
        dijkstra_table[col_index][row_index] = [[], dijkstra_table[col_index][row_index], 0]

dijkstra_table

,A,B,C,D,E,F,H,I
B,"[[], nan, 0]","[[], nan, 0]","[[], nan, 0]","[[], nan, 0]","[[], nan, 0]","[[], nan, 0]","[[], nan, 0]","[[], nan, 0]"
C,"[[], nan, 0]","[[], nan, 0]","[[], nan, 0]","[[], nan, 0]","[[], nan, 0]","[[], nan, 0]","[[], nan, 0]","[[], nan, 0]"
E,"[[], nan, 0]","[[], nan, 0]","[[], nan, 0]","[[], nan, 0]","[[], nan, 0]","[[], nan, 0]","[[], nan, 0]","[[], nan, 0]"
F,"[[], nan, 0]","[[], nan, 0]","[[], nan, 0]","[[], nan, 0]","[[], nan, 0]","[[], nan, 0]","[[], nan, 0]","[[], nan, 0]"
G,"[[], nan, 0]","[[], nan, 0]","[[], nan, 0]","[[], nan, 0]","[[], nan, 0]","[[], nan, 0]","[[], nan, 0]","[[], nan, 0]"
H,"[[], nan, 0]","[[], nan, 0]","[[], nan, 0]","[[], nan, 0]","[[], nan, 0]","[[], nan, 0]","[[], nan, 0]","[[], nan, 0]"
I,"[[], nan, 0]","[[], nan, 0]","[[], nan, 0]","[[], nan, 0]","[[], nan, 0]","[[], nan, 0]","[[], nan, 0]","[[], nan, 0]"
J,"[[], nan, 0]","[[], nan, 0]","[[], nan, 0]","[[], nan, 0]","[[], nan, 0]","[[], nan, 0]","[[], nan, 0]","[[], nan, 0]"


In the table above, each cell is a list that contains three information:
- **The path** that led us from the ```start_node``` to the node in the y-axis
- **The distance** between the ```start_node``` and the node in the y-axis
- 1 if the the node in the y-axis is a **leaf** (i.e. a dead end), 0 else

In [4]:
from_node = start_node
distance_to_add = 0
path_to_add = start_node

In [5]:
while from_node != end_node:

    n = 0

    # 1. Updating the Dijkstra table
    for elt in adjacency_table[from_node]: # start from the "from_node" and look at all the possible next steps in the adjacency table
        if not np.isnan(elt): # update the Dijkstra table if the node "elt" is a possible next step
            dijkstra_table[from_node][n][0] = path_to_add
            dijkstra_table[from_node][n][1] = distance_to_add + int(elt)
            dijkstra_table[from_node][n][2] = 1
        n = n + 1

    # 2. Finding the current best path in the Dijkstra table
    min_value = 100000000000 # taking a random super high value
    min_to = ""
    min_from = ""
    min_path = ""

    for row_index in dijkstra_table.index:
        for col_index in list(dijkstra_table):
            if dijkstra_table[col_index][row_index][2] == 1: # if the node is a leaf ...
                if dijkstra_table[col_index][row_index][1] < min_value: # ... and if its cumulated distance is inferior to the previous minimum ...
                    if row_index not in list(dijkstra_table) + ["J"]: # ... and if the node is a dead end ...
                        dijkstra_table[col_index][row_index][2] = 0 # ... do not take it as the new minimum
                    else: # ... if the node is not a dead end (still possible to go down in the graph after it) ...
                        min_path = dijkstra_table[col_index][row_index][0]
                        min_value = dijkstra_table[col_index][row_index][1]
                        min_to = row_index
                        min_from = col_index

    # 3. Removing the "leaf" qualification for the cells that are part of the new path
    if min_to != "J":
        dijkstra_table[min_from][min_to][2] = 0
    from_node = min_to
    distance_to_add = min_value
    path_to_add = [item for sublist in min_path for item in sublist] + [min_to]

dijkstra_table


,A,B,C,D,E,F,H,I
B,"[A, 85, 0]","[[], nan, 0]","[[], nan, 0]","[[], nan, 0]","[[], nan, 0]","[[], nan, 0]","[[], nan, 0]","[[], nan, 0]"
C,"[A, 217, 0]","[[], nan, 0]","[[], nan, 0]","[[], nan, 0]","[[], nan, 0]","[[], nan, 0]","[[], nan, 0]","[[], nan, 0]"
E,"[A, 173, 0]","[[], nan, 0]","[[], nan, 0]","[[], nan, 0]","[[], nan, 0]","[[], nan, 0]","[[], nan, 0]","[[], nan, 0]"
F,"[[], nan, 0]","[[A, B], 165, 0]","[[], nan, 0]","[[], nan, 0]","[[], nan, 0]","[[], nan, 0]","[[], nan, 0]","[[], nan, 0]"
G,"[[], nan, 0]","[[], nan, 0]","[[A, C], 403, 0]","[[], nan, 0]","[[], nan, 0]","[[], nan, 0]","[[], nan, 0]","[[], nan, 0]"
H,"[[], nan, 0]","[[], nan, 0]","[[A, C], 320, 0]","[[], nan, 0]","[[], nan, 0]","[[], nan, 0]","[[], nan, 0]","[[], nan, 0]"
I,"[[], nan, 0]","[[], nan, 0]","[[], nan, 0]","[[], nan, 0]","[[], nan, 0]","[[A, B, F], 415, 0]","[[], nan, 0]","[[], nan, 0]"
J,"[[], nan, 0]","[[], nan, 0]","[[], nan, 0]","[[], nan, 0]","[[A, E], 675, 1]","[[], nan, 0]","[[A, C, H], 487, 1]","[[A, B, F, I], 499, 1]"


And the shortest path is:

In [6]:
shortest_distance = 1000000
n = 0

for elt in dijkstra_table.loc["J",]:
    if str(elt[1]) != "nan":
        if elt[1] < shortest_distance:
            shortest_distance = elt[1]
            shortest_path = elt[0] + [dijkstra_table.index[n]]
    n = n + 1

print("Shortest path: ", shortest_path)
print("Shortest distance: ", shortest_distance)

Shortest path:  ['A', 'C', 'H', 'I']
Shortest distance:  487


# Food for thought

- **What to do if tie?** We should consider both paths. It is not because the paths have the same cumulated distances at an iteration i that they won't be different at i+1. With this algorithm, if there is a tie only the first path will be considered.

- **What happens if the start and end nodes are not connected?**